In [1]:
from gensim.models import Word2Vec, KeyedVectors
word2vec_model_path = './models/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
import pandas as pd

base_path = "./data/summary/"
write_path = "./data/prior_case/"

df=pd.read_csv(base_path+"data.csv")
df=df.head(1000)
sentences=df['sentences_english'].apply(eval)
print(" ".join(sentences[0]))
tokens=[word_tokenize(" ".join(i)) for i in sentences]
similarity_threshold = 0.5
keywords=['theft','rape','drug','murder','assault']
keywordVectors=[model[word] for word in keywords]

CrimeType=[]
for i in tokens:
    crimetype=-1
    for ind,keyvec in enumerate(keywordVectors):
        vectors=[]
        for word in i:
            try:
                word_vector = model[word]
                if (len(word_vector)):
                    vectors.append(word_vector)
            except:
                pass
        similarities = [cosine_similarity([keyvec], [vec])[0][0] for vec in vectors]
        contains_similar_words = any(similarity > similarity_threshold for similarity in similarities)
        if contains_similar_words:
            crimetype=ind
            break
    CrimeType.append(crimetype)

freqMap={}
for i in CrimeType:
    if i in freqMap:
        freqMap[i]+=1
    else:
        freqMap[i]=0

print(freqMap)

df['CrimeType']=CrimeType
df.to_csv(f"{write_path}/data_crime.csv",index=False)